In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 45.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 62.7 MB/s eta 0:00:00


Для задачи Question Answering мы используем класс BertForQuestionAnswering из библиотеки transformers.

In [6]:
import pandas as pd

import torch
from transformers import BertForQuestionAnswering
from transformers import BertTokenizer

Загрузим предъобученную модель BERT

In [3]:
model = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

Загрузим и токенайзер

In [4]:
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

In [7]:
data = pd.read_json("train.json")
data.tail(2)

,id,text,label,extracted_part
1797,903507299,ДОКУМЕНТАЦИЯ О ЗАКУПКЕ ЗАПРОС ПРЕДЛОЖЕНИЙ В ЭЛ...,обеспечение гарантийных обязательств,"{'text': [''], 'answer_start': [0], 'answer_en..."
1798,926039184,Государственная корпорацИя по атомной энергии ...,обеспечение гарантийных обязательств,"{'text': [''], 'answer_start': [0], 'answer_en..."


In [15]:
question = 'обеспечение гарантийных обязательств'
answer_text = data.to_string()

In [16]:
# Apply the tokenizer to the input text, treating them as a text-pair.
input_ids = tokenizer.encode(question, answer_text)

print('The input has a total of {:} tokens.'.format(len(input_ids)))

Token indices sequence length is longer than the specified maximum sequence length for this model (3657216 > 512). Running this sequence through the model will result in indexing errors


The input has a total of 3657216 tokens.


Посмотрим как выглядит процесс токенизации

In [ ]:
# BERT only needs the token IDs, but for the purpose of inspecting the 
# tokenizer's behavior, let's also get the token strings and display them.
tokens = tokenizer.convert_ids_to_tokens(input_ids)

# For each token and its id...
for token, id in zip(tokens, input_ids):
    
    # If this is the [SEP] token, add some space around it to make it stand out.
    if id == tokenizer.sep_token_id:
        print('')
    
    # Print the token string and its ID in two columns.
    print('{:<12} {:>6,}'.format(token, id))

    if id == tokenizer.sep_token_id:
        print('')

Выходные данные были обрезаны до нескольких последних строк (5000).
95            5,345
з             1,187
##а          10,260
##к          23,925
##о          14,150
##на         19,865
№             1,578
44            4,008
-             1,011
ф             1,199
##з          29,744
.             1,012
р             1,195
##е          15,290
##к          23,925
##в          25,529
##и          10,325
##з          29,744
##и          10,325
##т          22,919
##ы          29,113
с             1,196
##ч          29,752
##е          15,290
##т          22,919
##а          10,260
,             1,010
н             1,192
##а          10,260
к             1,189
##о          14,150
##т          22,919
##о          14,150
##р          16,856
##о          14,150
##м          29,745
в             1,182
с             1,196
##о          14,150
##о          14,150
##т          22,919
##в          25,529
##е          15,290
##т          22,919
##с          29,747
##т          22,919
##в         

Мы объединили question и answer_text, но BERT все еще должен как-то различать их. Для этого мы воспользуемся специальными Сегментными эмбеддингами ("Segment" embeddings), первый для сегманта "A", второй для "B".

In [ ]:
# Search the input_ids for the first instance of the `[SEP]` token.
sep_index = input_ids.index(tokenizer.sep_token_id)

# The number of segment A tokens includes the [SEP] token istelf.
num_seg_a = sep_index + 1

# The remainder are segment B.
num_seg_b = len(input_ids) - num_seg_a

# Construct the list of 0s and 1s.
segment_ids = [0]*num_seg_a + [1]*num_seg_b

# There should be a segment_id for every input token.
assert len(segment_ids) == len(input_ids)